In [1]:
import sparse
import torch
import sys
import os

from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from sb3_contrib.ppo_mask import MaskablePPO

from src.models.ppo_policy import CustomPPOPolicy
from src.utils.callback import CustomTensorboardCallback
from src.trainer import Trainer

os.chdir("..")

In [ ]:
trainer = Trainer('config.yaml')

trainer.envs[0].env

Using cuda device
saved_models/no_early_stopping_level1_model_3000000_steps.zip


In [5]:
import numpy as np
import torch
batch_size = 2
n_qubits = 4

batch_interactions = np.zeros((batch_size, n_qubits, n_qubits))
batch_interactions = torch.tensor(batch_interactions, dtype=torch.float32)
batch_interactions[0, 0, 1] = 1
batch_interactions[0, 1, 0] = 1
(batch_interactions != 0).nonzero(as_tuple=False)

tensor([[0, 0, 1],
        [0, 1, 0]])

In [6]:
vars(trainer.model.env.envs[0].env)


{'weights_reward': {'nonlocal': 10,
  'intervention': 20,
  'direct_capacity_violation': 30},
 'action_type': 'L',
 'mask_full_cores': True,
 'random_circuits': True,
 'gates_per_slice': [2],
 'n_qubits': 4,
 'n_cores': 2,
 'n_slices': 3,
 'observation_space': Dict('core_capacities': Box(0.0, inf, (2,), float32), 'flag': Box(0, 1, (4, 1), int32), 'interactions': Box(0, 1, (4, 4), int32), 'lookaheads': Box(0.0, inf, (4, 4), float32), 'n_qubits': Box(1, 4, (1,), int32), 'new_allocation': Box(0, 1, (4, 2), int32), 'old_allocation': Box(0, 1, (4, 2), int32)),
 'circuit': <COO: shape=(3, 4, 4), dtype=int64, nnz=12, fill_value=0>,
 'all_lookaheads': array([[[0.  , 0.25, 0.5 , 1.  ],
         [0.25, 0.  , 1.  , 0.5 ],
         [0.5 , 1.  , 0.  , 0.25],
         [1.  , 0.5 , 0.25, 0.  ]],
 
        [[0.  , 0.5 , 1.  , 0.  ],
         [0.5 , 0.  , 0.  , 1.  ],
         [1.  , 0.  , 0.  , 0.5 ],
         [0.  , 1.  , 0.5 , 0.  ]],
 
        [[0.  , 1.  , 0.  , 0.  ],
         [1.  , 0.  , 0.  , 

In [7]:
trainer.fit()

Logging to runs\prova_0
nlcomm computed in take action 1.0
nlcomm computed in take action 2.0
nlcomm computed in take action 1.0
nlcomm computed in take action 2.0
nlcomm computed in take action 2.0
Model saved at saved_models//prova_model_5_steps.zip
nlcomm computed in take action 2.0
-----------------------------------------------
| episode/                         |          |
|    direct_capacity_violation_sum | 0        |
|    final_reward                  | -220.0   |
|    intervention_sum              | 6        |
|    nl_comm_sum                   | 10       |
-----------------------------------------------
nlcomm computed in take action 1.0
nlcomm computed in take action 2.0
nlcomm computed in take action 2.0
nlcomm computed in take action 2.0
Model saved at saved_models//prova_model_10_steps.zip
nlcomm computed in take action 1.0
nlcomm computed in take action 2.0
-----------------------------------------------
| episode/                         |          |
|    direct_capac

In [8]:
# model

load_model = False
load_path = ''
config['policy_kwargs']['device'] = "cuda" if torch.cuda.is_available() else "cpu"

if load_model:
    model = MaskablePPO.load(load_path, env=env, device="cuda" if torch.cuda.is_available() else "cpu")

else:
    model = MaskablePPO(policy=CustomPPOPolicy,
                policy_kwargs=config['policy_kwargs'],
                env = env,
                device="cuda" if torch.cuda.is_available() else "cpu",
                verbose = 1,
                seed = 42,
                tensorboard_log = 'runs',
                **config['ppo']
            )

print(model.device)


NameError: name 'config' is not defined

In [ ]:
# train
model_path = 'models/'
run_id = 'test'

callback = CustomTensorboardCallback(save_path = model_path, verbose=1)    

model.learn(
    total_timesteps=config['total_timesteps'],
    callback=callback,
    reset_num_timesteps = False,
    tb_log_name=run_id
)